# SocGen Fraud Detection

In [1]:
import requests

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
from IPython.display import display, Javascript

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""

def load_ipython_extension(ip):
    display(Javascript(disable_js))
    print ("autoscrolling long output is disabled")

In [4]:
r = requests.get("https://he-s3.s3.amazonaws.com/media/hackathon/brainwaves-17-1/predict-fraud-transactions/f992303a-d-BW2017_2.zip")

In [5]:
print(r.headers)

In [6]:
import zipfile, io

In [7]:
z = zipfile.ZipFile(io.BytesIO(r.content))

In [8]:
z.extractall()

In [9]:
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
os.listdir()

['test1.csv',
 'fraud_detection.ipynb',
 'test.csv',
 'sample_submission.csv',
 'result.csv',
 'result_1.csv',
 'sample_submissions.csv',
 'train.csv',
 'f992303a-d-BW2017_2.zip',
 'train1.csv',
 'return_pred.ipynb',
 '.ipynb_checkpoints']

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier, LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, confusion_matrix
#pd.options.display.precision =  4
import matplotlib.pyplot as plt

%matplotlib inline

### Reading input datasets

In [12]:
dev_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [13]:
display(dev_data.head())
display(test_data.head())

,transaction_id,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_1,cat_var_2,...,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,target
0,id_11,2.302632e-08,0.040182,0.0,1.800000e-07,2.302632e-08,2.368421e-08,1.115205e-08,NaN,ce,...,0,0,0,0,0,0,0,0,0,0
1,id_33,7.965789e-06,0.157872,0.0,2.105000e-06,2.769737e-07,7.965789e-06,2.433058e-06,da,tn,...,0,0,0,0,0,0,0,0,0,0
2,id_51,7.828947e-08,0.089140,0.0,3.550000e-07,4.671053e-08,1.052632e-07,4.276014e-07,gf,ce,...,0,0,0,0,0,0,0,0,0,0
3,id_54,7.894737e-08,0.227239,0.0,1.050000e-06,1.381579e-07,2.190789e-07,1.848054e-08,NaN,ce,...,0,0,0,0,0,0,0,0,0,0
4,id_62,3.321053e-06,0.160410,0.0,2.105000e-06,2.769737e-07,3.340789e-06,2.152983e-06,da,tn,...,0,0,0,0,0,0,0,0,0,0


,transaction_id,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_1,cat_var_2,...,cat_var_33,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42
0,id_1,4.736842e-07,0.162737,0.0,2.105000e-06,2.769737e-07,4.828947e-07,1.300010e-07,da,tn,...,0,0,0,0,0,0,0,0,0,0
1,id_6,6.578947e-08,0.086391,0.0,7.500000e-07,1.315789e-07,9.868421e-08,3.186300e-08,pu,ce,...,0,0,0,0,0,0,0,0,0,0
2,id_9,1.585526e-07,0.159987,0.0,2.105000e-06,2.769737e-07,1.585526e-07,4.715724e-08,da,tn,...,0,0,0,0,0,0,0,0,0,0
3,id_14,8.552632e-08,0.088083,0.0,7.500000e-07,8.552632e-08,9.868421e-08,4.142190e-08,pu,ce,...,0,0,0,0,0,0,0,0,0,0
4,id_15,1.144737e-07,0.089563,0.0,3.550000e-07,4.671053e-08,4.407895e-08,1.452953e-07,gf,ce,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#pd.options.display.float_format = '{:.10f}'.format
display(dev_data.describe())
display(test_data.describe())


,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_19,cat_var_20,cat_var_21,...,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,target
count,3.489780e+05,348978.000000,348978.000000,3.489780e+05,3.489780e+05,3.489780e+05,3.489780e+05,348978.000000,348978.000000,348978.000000,...,348978.000000,348978.0,348978.0,348978.0,348978.0,348978.000000,348978.0,348978.000000,348978.0,348978.000000
mean,2.059731e-05,0.160586,0.000011,4.604324e-05,8.187931e-06,1.482768e-05,1.942554e-05,0.520279,0.479721,0.672257,...,0.000003,0.0,0.0,0.0,0.0,0.000009,0.0,0.000003,0.0,0.107078
std,1.930948e-03,0.131499,0.002538,1.999947e-03,7.213736e-04,1.492990e-03,1.462171e-03,0.499589,0.499589,0.469391,...,0.001693,0.0,0.0,0.0,0.0,0.002932,0.0,0.001693,0.0,0.309213
min,0.000000e+00,0.000317,0.000000,4.000000e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
25%,4.605263e-08,0.084514,0.000000,3.550000e-07,4.671053e-08,4.407895e-08,1.720602e-08,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
50%,1.802632e-07,0.101512,0.000000,1.875000e-06,2.598684e-07,9.868421e-08,8.252516e-08,1.000000,0.000000,1.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
75%,6.513158e-07,0.160833,0.000000,2.105000e-06,2.769737e-07,4.618421e-07,3.571842e-07,1.000000,1.000000,1.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
max,5.427632e-01,1.000000,0.758621,3.750000e-01,2.171053e-01,4.605263e-01,3.542030e-01,1.000000,1.000000,1.000000,...,1.000000,0.0,0.0,0.0,0.0,1.000000,0.0,1.000000,0.0,1.000000


,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_19,cat_var_20,cat_var_21,...,cat_var_33,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42
count,5.234660e+05,523466.000000,523466.000000,5.234660e+05,5.234660e+05,5.234660e+05,5.234660e+05,523466.000000,523466.000000,523466.000000,...,523466.000000,523466.000000,523466.000000,523466.000000,523466.000000,523466.0,523466.000000,523466.000000,523466.0,523466.0
mean,1.740233e-05,0.160848,0.000022,4.311262e-05,6.943160e-06,1.551274e-05,2.252647e-05,0.522489,0.477511,0.671914,...,0.000008,0.000006,0.000006,0.000002,0.000002,0.0,0.000006,0.000004,0.0,0.0
std,1.643428e-03,0.131686,0.003843,2.692392e-03,5.174122e-04,1.540989e-03,2.230268e-03,0.499494,0.499494,0.469517,...,0.002764,0.002394,0.002394,0.001382,0.001382,0.0,0.002394,0.001955,0.0,0.0
min,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
25%,4.671053e-08,0.084911,0.000000,3.550000e-07,4.671053e-08,4.407895e-08,1.720602e-08,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
50%,1.815789e-07,0.104473,0.000000,1.875000e-06,2.611842e-07,9.868421e-08,8.348105e-08,1.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
75%,6.513158e-07,0.160833,0.000000,2.105000e-06,2.769737e-07,4.664474e-07,3.575028e-07,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
max,5.427632e-01,1.000000,1.000000,1.000000e+00,2.171053e-01,4.605263e-01,1.000000e+00,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.0,0.0


In [15]:
display(dev_data.info())
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348978 entries, 0 to 348977
Data columns (total 51 columns):
transaction_id    348978 non-null object
num_var_1         348978 non-null float64
num_var_2         348978 non-null float64
num_var_3         348978 non-null float64
num_var_4         348978 non-null float64
num_var_5         348978 non-null float64
num_var_6         348978 non-null float64
num_var_7         348978 non-null float64
cat_var_1         333196 non-null object
cat_var_2         348978 non-null object
cat_var_3         305125 non-null object
cat_var_4         348978 non-null object
cat_var_5         348978 non-null object
cat_var_6         348978 non-null object
cat_var_7         348978 non-null object
cat_var_8         239240 non-null object
cat_var_9         348978 non-null object
cat_var_10        348978 non-null object
cat_var_11        348978 non-null object
cat_var_12        348978 non-null object
cat_var_13        348978 non-null object
cat_var_14        348

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523466 entries, 0 to 523465
Data columns (total 50 columns):
transaction_id    523466 non-null object
num_var_1         523466 non-null float64
num_var_2         523466 non-null float64
num_var_3         523466 non-null float64
num_var_4         523466 non-null float64
num_var_5         523466 non-null float64
num_var_6         523466 non-null float64
num_var_7         523466 non-null float64
cat_var_1         504774 non-null object
cat_var_2         523466 non-null object
cat_var_3         470104 non-null object
cat_var_4         523466 non-null object
cat_var_5         523466 non-null object
cat_var_6         501523 non-null object
cat_var_7         523466 non-null object
cat_var_8         515328 non-null object
cat_var_9         523466 non-null object
cat_var_10        523466 non-null object
cat_var_11        523466 non-null object
cat_var_12        523466 non-null object
cat_var_13        523466 non-null object
cat_var_14        523

### Checking missing values in columns

In [16]:
display((dev_data.isnull().sum()/dev_data.isnull().count()).sort_values(ascending=False))
(test_data.isnull().sum()/test_data.isnull().count()).sort_values(ascending=False)

cat_var_8         0.314455
cat_var_3         0.125661
cat_var_1         0.045223
target            0.000000
cat_var_6         0.000000
cat_var_15        0.000000
cat_var_14        0.000000
cat_var_13        0.000000
cat_var_12        0.000000
cat_var_11        0.000000
cat_var_10        0.000000
cat_var_9         0.000000
cat_var_7         0.000000
cat_var_4         0.000000
cat_var_5         0.000000
cat_var_17        0.000000
cat_var_2         0.000000
num_var_7         0.000000
num_var_6         0.000000
num_var_5         0.000000
num_var_4         0.000000
num_var_3         0.000000
num_var_2         0.000000
num_var_1         0.000000
cat_var_16        0.000000
cat_var_18        0.000000
cat_var_42        0.000000
cat_var_31        0.000000
cat_var_41        0.000000
cat_var_40        0.000000
cat_var_39        0.000000
cat_var_38        0.000000
cat_var_37        0.000000
cat_var_36        0.000000
cat_var_35        0.000000
cat_var_34        0.000000
cat_var_33        0.000000
c

cat_var_3         0.101940
cat_var_6         0.041919
cat_var_1         0.035708
cat_var_8         0.015546
cat_var_42        0.000000
cat_var_15        0.000000
cat_var_14        0.000000
cat_var_13        0.000000
cat_var_12        0.000000
cat_var_11        0.000000
cat_var_10        0.000000
cat_var_9         0.000000
cat_var_7         0.000000
cat_var_4         0.000000
cat_var_5         0.000000
cat_var_41        0.000000
cat_var_2         0.000000
num_var_7         0.000000
num_var_6         0.000000
num_var_5         0.000000
num_var_4         0.000000
num_var_3         0.000000
num_var_2         0.000000
num_var_1         0.000000
cat_var_16        0.000000
cat_var_17        0.000000
cat_var_18        0.000000
cat_var_19        0.000000
cat_var_40        0.000000
cat_var_39        0.000000
cat_var_38        0.000000
cat_var_37        0.000000
cat_var_36        0.000000
cat_var_35        0.000000
cat_var_34        0.000000
cat_var_33        0.000000
cat_var_32        0.000000
c

In [17]:
missing_data = ['cat_var_8', 'cat_var_3']

In [18]:
dev_data.drop(missing_data, axis=1, inplace=True)
test_data.drop(missing_data, axis=1, inplace=True)
mode_var1 = dev_data['cat_var_1'].value_counts().index[0]
mode_var6 = dev_data['cat_var_6'].value_counts().index[0]
print(mode_var1)
print(mode_var6)
dev_data['cat_var_1'].fillna(mode_var1, inplace=True)
test_data['cat_var_1'].fillna(mode_var1, inplace=True)
dev_data['cat_var_6'].fillna(mode_var6, inplace=True)
test_data['cat_var_6'].fillna(mode_var6, inplace=True)


gf
zs


In [19]:
display(dev_data.isnull().values.any())
display(test_data.isnull().values.any())

False

False

In [20]:
cat_columns = dev_data.select_dtypes(['object']).columns
cat_columns

Index(['transaction_id', 'cat_var_1', 'cat_var_2', 'cat_var_4', 'cat_var_5',
       'cat_var_6', 'cat_var_7', 'cat_var_9', 'cat_var_10', 'cat_var_11',
       'cat_var_12', 'cat_var_13', 'cat_var_14', 'cat_var_15', 'cat_var_16',
       'cat_var_17', 'cat_var_18'],
      dtype='object')

In [21]:
cat_columns = cat_columns[1:]

In [22]:
display(dev_data[cat_columns].describe())
display(test_data[cat_columns].describe())

,cat_var_1,cat_var_2,cat_var_4,cat_var_5,cat_var_6,cat_var_7,cat_var_9,cat_var_10,cat_var_11,cat_var_12,cat_var_13,cat_var_14,cat_var_15,cat_var_16,cat_var_17,cat_var_18
count,348978,348978,348978,348978,348978,348978,348978,348978,348978,348978,348978,348978,348978,348978,348978,348978
unique,534,60,2,2,518,20,5,23,5,5,52,12,2,2,2,2
top,gf,ce,tn,tn,zs,ep,ep,ye,ce,tn,hr,db,ep,tn,tn,ep
freq,147952,233079,232909,174898,132059,348539,81529,34614,82301,142790,132770,248013,236413,320148,254043,199168


,cat_var_1,cat_var_2,cat_var_4,cat_var_5,cat_var_6,cat_var_7,cat_var_9,cat_var_10,cat_var_11,cat_var_12,cat_var_13,cat_var_14,cat_var_15,cat_var_16,cat_var_17,cat_var_18
count,523466,523466,523466,523466,523466,523466,523466,523466,523466,523466,523466,523466,523466,523466,523466,523466
unique,534,62,2,2,515,22,5,23,5,5,52,12,2,2,2,2
top,gf,ce,tn,ep,zs,ep,ep,ye,ce,tn,hr,db,ep,tn,tn,ep
freq,217285,348977,350336,262085,220366,522824,122473,52252,123097,214360,199583,372861,354042,480857,380330,298498


In [ ]:
dev_data.corr()

In [23]:
print(set(dev_data['cat_var_2'].value_counts().index) - set(test_data['cat_var_2'].value_counts().index))
print(set(dev_data['cat_var_6'].value_counts().index) - set(test_data['cat_var_6'].value_counts().index))
print(set(dev_data['cat_var_7'].value_counts().index) - set(test_data['cat_var_7'].value_counts().index))
print('#-------------------------------------------------------------------------------------------------------')
print(set(test_data['cat_var_2'].value_counts().index) - set(dev_data['cat_var_2'].value_counts().index))
print(set(test_data['cat_var_6'].value_counts().index) - set(dev_data['cat_var_6'].value_counts().index))
print(set(test_data['cat_var_7'].value_counts().index) - set(dev_data['cat_var_7'].value_counts().index))

{'tu'}
{'qw', 'ln', 'qt'}
{'ts'}
#-------------------------------------------------------------------------------------------------------
{'uq', 'on', 'op'}
set()
{'cg', 'xy', 'mm'}


### Getting dummy variables

In [24]:
for col in cat_columns:
    dev_data = pd.concat([dev_data,pd.get_dummies(dev_data[col], prefix=col, drop_first=True).astype(np.int8)], axis=1)
    dev_data.drop(col, axis=1, inplace=True)
    test_data = pd.concat([test_data, pd.get_dummies(test_data[col], prefix=col, drop_first=True).astype(np.int8)], axis=1)
    test_data.drop(col, axis=1, inplace=True)

In [25]:
display(dev_data.columns[0:50])
test_data.columns[0:50]

Index(['transaction_id', 'num_var_1', 'num_var_2', 'num_var_3', 'num_var_4',
       'num_var_5', 'num_var_6', 'num_var_7', 'cat_var_19', 'cat_var_20',
       'cat_var_21', 'cat_var_22', 'cat_var_23', 'cat_var_24', 'cat_var_25',
       'cat_var_26', 'cat_var_27', 'cat_var_28', 'cat_var_29', 'cat_var_30',
       'cat_var_31', 'cat_var_32', 'cat_var_33', 'cat_var_34', 'cat_var_35',
       'cat_var_36', 'cat_var_37', 'cat_var_38', 'cat_var_39', 'cat_var_40',
       'cat_var_41', 'cat_var_42', 'target', 'cat_var_1_ab', 'cat_var_1_ac',
       'cat_var_1_ad', 'cat_var_1_ae', 'cat_var_1_af', 'cat_var_1_ag',
       'cat_var_1_ah', 'cat_var_1_aj', 'cat_var_1_al', 'cat_var_1_am',
       'cat_var_1_ao', 'cat_var_1_ap', 'cat_var_1_aq', 'cat_var_1_as',
       'cat_var_1_at', 'cat_var_1_au', 'cat_var_1_av'],
      dtype='object')

Index(['transaction_id', 'num_var_1', 'num_var_2', 'num_var_3', 'num_var_4',
       'num_var_5', 'num_var_6', 'num_var_7', 'cat_var_19', 'cat_var_20',
       'cat_var_21', 'cat_var_22', 'cat_var_23', 'cat_var_24', 'cat_var_25',
       'cat_var_26', 'cat_var_27', 'cat_var_28', 'cat_var_29', 'cat_var_30',
       'cat_var_31', 'cat_var_32', 'cat_var_33', 'cat_var_34', 'cat_var_35',
       'cat_var_36', 'cat_var_37', 'cat_var_38', 'cat_var_39', 'cat_var_40',
       'cat_var_41', 'cat_var_42', 'cat_var_1_ab', 'cat_var_1_ac',
       'cat_var_1_ad', 'cat_var_1_ae', 'cat_var_1_af', 'cat_var_1_ag',
       'cat_var_1_ah', 'cat_var_1_aj', 'cat_var_1_al', 'cat_var_1_am',
       'cat_var_1_ao', 'cat_var_1_ap', 'cat_var_1_aq', 'cat_var_1_as',
       'cat_var_1_at', 'cat_var_1_au', 'cat_var_1_av', 'cat_var_1_aw'],
      dtype='object')

In [26]:
remove_from_test = list(set(test_data.columns)-set(dev_data.columns))
remove_from_dev = list((set(dev_data.columns)-set(test_data.columns))-set(['target']))
print(remove_from_test)
print(remove_from_dev)

['cat_var_7_mm', 'cat_var_2_op', 'cat_var_2_uq', 'cat_var_7_xy', 'cat_var_7_cg', 'cat_var_1_yw', 'cat_var_2_on', 'cat_var_1_jk', 'cat_var_1_gz']
['cat_var_7_ts', 'cat_var_6_qt', 'cat_var_6_qw', 'cat_var_1_ye', 'cat_var_6_ln', 'cat_var_1_ln', 'cat_var_2_tu', 'cat_var_1_ex']


In [27]:
dev_data.drop(remove_from_dev, axis=1, inplace=True)
test_data.drop(remove_from_test, axis=1, inplace=True)

In [28]:
remove_from_test = list(set(test_data.columns)-set(dev_data.columns))
remove_from_dev = list(set(dev_data.columns)-set(test_data.columns))
print(remove_from_test)
print(remove_from_dev)

[]
['target']


### Resampling to fix class imbalance

In [29]:
dev_data['target'].value_counts()

0    311610
1     37368
Name: target, dtype: int64

In [30]:
# Separate majority and minority classes
dev_data_0 = dev_data[dev_data.target==0]
dev_data_1 = dev_data[dev_data.target==1]

In [31]:
from sklearn.utils import resample

In [32]:
#Upsample minority class
dev_data_1_upsampled = resample(dev_data_1, 
                                 replace=True,     # sample with replacement
                                 n_samples=311610,    # to match majority class
                                 random_state=123) # reproducible results
#Downsample majority class
dev_data_0_downsampled = resample(dev_data_0, 
                                 replace=False,     # sample with replacement
                                 n_samples=37368,    # to match majority class
                                 random_state=123) # reproducible results


In [33]:
dev_data_upsampled = pd.concat([dev_data_0, dev_data_1_upsampled])
dev_data_downsampled = pd.concat([dev_data_0_downsampled, dev_data_1])

In [34]:
print(len(dev_data_upsampled))
len(dev_data_downsampled)

623220


74736

In [35]:
dev_data_downsampled.iloc[:,3].describe()

count    7.473600e+04
mean     9.227884e-07
std      1.783812e-04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.448276e-02
Name: num_var_3, dtype: float64

In [37]:
msk = np.random.rand(len(dev_data_upsampled)) < 0.8

In [38]:
labels = dev_data_downsampled['target'].copy()
features = dev_data_downsampled.drop(['target', 'transaction_id'], axis=1)
#train = dev_data_upsampled[msk]
#valid = dev_data_upsampled[~msk]

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2)
X_test = test_data.drop(['transaction_id'], axis=1)

In [41]:
print(labels.value_counts())

1    37368
0    37368
Name: target, dtype: int64


In [42]:
print(X_train.shape)
print(X_test.shape)
X_valid.shape

(59788, 1253)
(523466, 1253)


(14948, 1253)

In [43]:
(X_test).iloc[:,1:]

,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_19,cat_var_20,cat_var_21,cat_var_22,...,cat_var_14_ip,cat_var_14_iq,cat_var_14_ji,cat_var_14_rv,cat_var_14_td,cat_var_14_tn,cat_var_15_tn,cat_var_16_tn,cat_var_17_tn,cat_var_18_tn
0,0.162737,0.0,2.105000e-06,2.769737e-07,4.828947e-07,1.300010e-07,1,0,1,0,...,0,0,0,0,0,0,1,1,1,1
1,0.086391,0.0,7.500000e-07,1.315789e-07,9.868421e-08,3.186300e-08,0,1,1,0,...,0,0,0,0,0,0,0,0,1,1
2,0.159987,0.0,2.105000e-06,2.769737e-07,1.585526e-07,4.715724e-08,1,0,1,0,...,0,0,0,0,0,0,1,1,0,1
3,0.088083,0.0,7.500000e-07,8.552632e-08,9.868421e-08,4.142190e-08,0,1,1,0,...,0,0,0,0,0,0,0,0,1,1
4,0.089563,0.0,3.550000e-07,4.671053e-08,4.407895e-08,1.452953e-07,0,1,0,1,...,0,0,0,0,0,0,0,1,1,0
5,0.582214,0.0,7.680000e-06,2.631579e-07,1.485526e-06,2.048791e-07,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
6,0.293751,0.0,5.865000e-06,6.355263e-07,1.578947e-07,7.647119e-08,1,0,1,0,...,1,0,0,0,0,0,0,1,1,0
7,0.162737,0.0,2.105000e-06,2.769737e-07,1.348684e-07,9.558899e-08,1,0,1,0,...,0,0,0,0,0,0,1,1,0,1
8,0.494026,0.0,2.890000e-06,3.407895e-07,3.730263e-07,7.647119e-09,0,1,1,0,...,0,0,0,0,0,0,0,0,1,1
9,0.078355,0.0,3.550000e-07,4.671053e-08,4.407895e-08,4.938765e-08,0,1,0,1,...,0,0,0,0,0,0,0,1,1,0


### PCA: Feature selection

In [44]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
pca = PCA(n_components=.99, svd_solver='full')
X_train = pca.fit_transform(X_train)
X_valid = pca.transform(X_valid)
X_test=  pca.transform(X_test)
print(pca.n_components_)
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

348
(59788, 348)
(14948, 348)
(523466, 348)


In [ ]:
#pca = PCA(n_components=0.99, svd_solver='full')
#X_train = pca.fit_transform(X_train)
#X_valid = pca.transform(X_valid)
#pca.n_components_

In [45]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)
X_test = sc.transform(X_test)

In [ ]:
sc.mean_

### Logistic Regression

In [46]:
from sklearn.linear_model import LogisticRegression

In [47]:
cls = LogisticRegression()
cls.fit(X_train, y_train)
print('accuracy', cls.score(X_valid, y_valid))
y_pred = cls.predict(X_valid)
print('roc score', roc_auc_score(y_valid, y_pred))
probs = cls.predict_proba(X_test)
probs = np.around(probs, decimals=6)

accuracy 0.643765052181
roc score 0.645359000077


In [ ]:
confusion_matrix(y_valid, y_pred)

In [ ]:
# Decision Tree Classifier
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
# load the iris datasets
# fit a CART model to the data
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
print(model)
# make predictions
expected = y_valid
predicted = model.predict(X_valid)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

In [ ]:
print('roc score', roc_auc_score(expected, predicted))
probs = model.predict_proba(X_test)

### ANN

In [49]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras.backend as k
import keras

Using TensorFlow backend.


In [69]:
k.clear_session()
model = Sequential()
model.add(Dense(100, input_shape=(348,), kernel_initializer="uniform", activation='relu'))
model.add(Dense(100, kernel_initializer="uniform", activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer="uniform"))
#model.summary()
# 4
OPTIM = keras.optimizers.Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
model.summary()

tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', write_graph=True)
model.fit(X_train,y_train, epochs=200, batch_size=1000,  verbose=1, validation_split=0.2, callbacks=[tbCallBack])
scores = model.evaluate(X_valid, y_valid)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
predicted = model.predict_classes(X_valid)
expected = y_valid
print('roc score', roc_auc_score(expected, predicted))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               34900     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 45,101
Trainable params: 45,101
Non-trainable params: 0
_________________________________________________________________
Train on 47830 samples, validate on 11958 samples
Epoch 1/200
47830/47830 [==============================] - 0s 9us/step - loss: 1.7267 - acc: 0.5022 - val_loss: 1.3248 - val_acc: 0.4960
Epoch 2/200
47830/47830 [==============================] - 0s 9us/step - loss: 1.19

Epoch 54/200
47830/47830 [==============================] - 0s 8us/step - loss: 0.6048 - acc: 0.6410 - val_loss: 0.6234 - val_acc: 0.6341
Epoch 55/200
47830/47830 [==============================] - 0s 9us/step - loss: 0.6074 - acc: 0.6431 - val_loss: 0.6208 - val_acc: 0.6361
Epoch 56/200
47830/47830 [==============================] - 0s 8us/step - loss: 0.6035 - acc: 0.6404 - val_loss: 0.6191 - val_acc: 0.6397
Epoch 57/200
47830/47830 [==============================] - 0s 9us/step - loss: 0.6019 - acc: 0.6399 - val_loss: 0.6187 - val_acc: 0.6397
Epoch 58/200
47830/47830 [==============================] - 0s 9us/step - loss: 0.6030 - acc: 0.6364 - val_loss: 0.6171 - val_acc: 0.6396
Epoch 59/200
47830/47830 [==============================] - 0s 8us/step - loss: 0.6005 - acc: 0.6395 - val_loss: 0.6154 - val_acc: 0.6398
Epoch 60/200
47830/47830 [==============================] - 0s 8us/step - loss: 0.6017 - acc: 0.6398 - val_loss: 0.6151 - val_acc: 0.6397
Epoch 61/200
47830/47830 [========

47830/47830 [==============================] - 0s 9us/step - loss: 0.5849 - acc: 0.6128 - val_loss: 0.6193 - val_acc: 0.5731
Epoch 114/200
47830/47830 [==============================] - 0s 9us/step - loss: 0.5838 - acc: 0.6078 - val_loss: 0.6195 - val_acc: 0.5656
Epoch 115/200
47830/47830 [==============================] - 0s 8us/step - loss: 0.5842 - acc: 0.6092 - val_loss: 0.6194 - val_acc: 0.5664
Epoch 116/200
47830/47830 [==============================] - 0s 8us/step - loss: 0.5834 - acc: 0.6057 - val_loss: 0.6195 - val_acc: 0.5610
Epoch 117/200
47830/47830 [==============================] - 0s 8us/step - loss: 0.5824 - acc: 0.6043 - val_loss: 0.6202 - val_acc: 0.5577
Epoch 118/200
47830/47830 [==============================] - 0s 9us/step - loss: 0.5844 - acc: 0.6033 - val_loss: 0.6201 - val_acc: 0.5585
Epoch 119/200
47830/47830 [==============================] - 0s 9us/step - loss: 0.5825 - acc: 0.6046 - val_loss: 0.6194 - val_acc: 0.5573
Epoch 120/200
47830/47830 [==============

47830/47830 [==============================] - 0s 9us/step - loss: 0.5751 - acc: 0.5792 - val_loss: 0.6568 - val_acc: 0.5442
Epoch 173/200
47830/47830 [==============================] - 0s 8us/step - loss: 0.5730 - acc: 0.5803 - val_loss: 0.6538 - val_acc: 0.5440
Epoch 174/200
47830/47830 [==============================] - 0s 9us/step - loss: 0.5744 - acc: 0.5821 - val_loss: 0.6540 - val_acc: 0.5430
Epoch 175/200
47830/47830 [==============================] - 0s 9us/step - loss: 0.5718 - acc: 0.5823 - val_loss: 0.6588 - val_acc: 0.5442
Epoch 176/200
47830/47830 [==============================] - 0s 9us/step - loss: 0.5707 - acc: 0.5801 - val_loss: 0.6557 - val_acc: 0.5440
Epoch 177/200
47830/47830 [==============================] - 0s 10us/step - loss: 0.5716 - acc: 0.5814 - val_loss: 0.6599 - val_acc: 0.5423
Epoch 178/200
47830/47830 [==============================] - 0s 9us/step - loss: 0.5686 - acc: 0.5800 - val_loss: 0.6578 - val_acc: 0.5434
Epoch 179/200
47830/47830 [=============

In [67]:
confusion_matrix(expected, predicted)

array([[6896,  529],
       [4804, 2719]])

### Writing submission file

In [68]:
with open('result.csv', 'w') as f:
    f.write('transaction_id, target\n')
    for i in range(len(probs)):
        f.write(str(test_data.iloc[i, 0])+ ','+ str(probs[i, 1])+'\n')
        